# Convert JSON output to BRAT 

This is an experimental version to convert the custom JSON data back to BRAT format

It is based on heuristics

In [1]:
# Load libraris
import ast
import json
import pathlib
import re
import tqdm
from collections import defaultdict, OrderedDict, Counter
from typing import List, Dict, Tuple, Set

import pandas as pd

In [2]:
# Original n2c2 2018 track 2 data folder
BASE_DATA_DIR = pathlib.Path("/prj/doctoral_letters/data/i2b2_2018track2/test")

# JSON data folder
CSV_PATH      = pathlib.Path("/home/prichter/research/MIEQA/output_results_llama3_8b_i2b2_pydantic.csv")

# Where to store the converted output
OUT_DIR       = pathlib.Path("json2brat")
OUT_DIR.mkdir(exist_ok=True, parents=True)
ANN_DIR    = pathlib.Path("json2brat/system")
ANN_DIR.mkdir(exist_ok=True, parents=True)

# Define relation information keys
ENTITY_KEYS = ["strength","dosage","form","frequency","route","reason","ade","duration"]

In [3]:
# Helper functions

# Split the JSON data into separate letters
_ADM_RE = re.compile(
    r"Admission\s+Date:\s*\[\*\*([0-9]{4}-[0-9]{1,2}-[0-9]{1,2})\*\*\]",
    re.I,
)

def _build_letter_map(csv_path: pathlib.Path) -> Dict[str, List[dict]]:
    """Map lines back to distinct letters marked by admission date"""
    df = pd.read_csv(csv_path, sep="|", names=["text", "gold", "pred"], header=0)

    letter_map: Dict[str, List[dict]] = defaultdict(list)
    current_token = None

    for _, row in df.iterrows():
        text_line: str = str(row["text"])
        m = _ADM_RE.search(text_line)
        if m:
            current_token = f"[**{m.group(1)}**]"

        if current_token is None:
            continue

        try:
            pred_od = eval(row["pred"])
            meds = pred_od.get("medications", [])
            letter_map[current_token].extend(meds)
        except Exception:
            continue

    return letter_map

LETTER_PRED_MAP = _build_letter_map(CSV_PATH)

def load_note(letter_id: str) -> str:
    return (BASE_DATA_DIR / f"{letter_id}.txt").read_text(encoding="utf-8")

def extract_admission_token(note_text: str) -> str:
    m = _ADM_RE.search(note_text)
    if not m:
        raise ValueError("Admission Date token not found in original note")
    return f"[**{m.group(1)}**]"

def normalise_spaces(s: str) -> str:
    import re
    return re.sub(r"\s+", " ", s.strip())

def find_offset(text: str, substring: str, used_spans: Set[Tuple[int, int]],
                start_pos: int = 0) -> Tuple[int, int]:
    if not substring:
        raise ValueError("Empty substring")
    while True:
        idx = text.find(substring, start_pos)
        if idx == -1:
            break
        span = (idx, idx + len(substring))
        if all(not (max(a, span[0]) < min(b, span[1])) for a, b in used_spans):
            return span
        start_pos = idx + 1
    raise ValueError(f"Could not locate unique span for: {substring!r}")

def parse_ann_to_counters(path):
    """
    Return two Counters:
      • ent_counter[(label,text)]
      • rel_counter[(relType,'','')]
    """
    ent_counter = Counter()
    rel_counter = Counter()

    with open(path, encoding="utf-8") as fh:
        for line in fh:
            if line.startswith("T"):
                parts = line.rstrip("\n").split("\t")
                if len(parts) != 3:
                    continue         
                label = parts[1].split()[0]
                text  = parts[2]
                ent_counter[(label, text)] += 1

            elif line.startswith("R"):
                parts = line.rstrip("\n").split("\t")
                if len(parts) != 2:
                    continue         
                rel_type = parts[1].split()[0]
                rel_counter[(rel_type, "", "")] += 1

    return ent_counter, rel_counter

# Create an entity type map
ENTITY_MAP = {
    "medication": ("Drug", "Drug"),
    "strength":   ("Strength", "Strength-Drug"),
    "dosage":     ("Dosage", "Dosage-Drug"),
    "form":       ("Form", "Form-Drug"),
    "frequency":  ("Frequency", "Frequency-Drug"),
    "route":      ("Route", "Route-Drug"),
    "reason":     ("Reason", "Reason-Drug"),
    "ade":        ("ADE", "ADE-Drug"),
    "duration":   ("Duration", "Duration-Drug"),
}

def convert_one_letter(letter_id: str) -> pathlib.Path:
    """
    Produces .ann BRAT files from JSON data
    """
    note_text = load_note(letter_id)
    adm_token = extract_admission_token(note_text)

    meds = LETTER_PRED_MAP.get(adm_token, [])
    if not meds:
        raise ValueError(f"No prediction rows found for admission token {adm_token}")

    ann_lines: List[str] = []
    used: Set[Tuple[int, int]] = set()
    tid = rid = 1

    def next_tid():
        nonlocal tid
        tid += 1
        return f"T{tid-1}"

    def next_rid():
        nonlocal rid
        rid += 1
        return f"R{rid-1}"

    for med in meds:
        drug_name_raw = normalise_spaces(med.get("medication", ""))
        drug_name = re.sub(r"\s*\(\d+\)\s*$", "", drug_name_raw)
        try:
            s, e = find_offset(note_text, drug_name, used)
        except ValueError:
            continue
        drug_tid = next_tid()
        ann_lines.append(f"{drug_tid}\tDrug {s} {e}\t{drug_name}")
        used.add((s, e))

        for key, val in med.items():
            if key == "medication" or not val:
                continue
            vals = val if isinstance(val, list) else [val]
            for v in vals:
                v_norm = normalise_spaces(v)
                try:
                    s2, e2 = find_offset(note_text, v_norm, used)
                except ValueError:
                    continue
                ent_type, rel_type = ENTITY_MAP[key]
                ent_tid = next_tid()
                ann_lines.append(f"{ent_tid}\t{ent_type} {s2} {e2}\t{v_norm}")
                used.add((s2, e2))
                ann_lines.append(f"{next_rid()}\t{rel_type} Arg1:{ent_tid} Arg2:{drug_tid}")

    out_path = OUT_DIR / f"system/{letter_id}.ann"
    out_path.write_text("\n".join(ann_lines), encoding="utf-8")
    return out_path

In [4]:
## Convert JSON data to BRAT files
outs = []
for txt_path in tqdm.tqdm(BASE_DATA_DIR.glob("*.txt")):
    letter_id = txt_path.stem
    outs.append(convert_one_letter(letter_id))

print(f"Wrote {len(outs)} .ann files to {OUT_DIR}")

202it [00:04, 47.36it/s]

Wrote 202 .ann files to json2brat


## Validation

In [5]:
gold_ent      = Counter() 
gold_rel      = Counter()
gold_ent_cls  = Counter()
gold_rel_cls  = Counter()

# Parse all JSON lines
df = pd.read_csv(CSV_PATH, sep="|", names=["text","gold","pred"], header=1)
for od in (eval(g) for g in df["gold"]):
    for m in od.get("medications", []):
        drug = m.get("medication","").strip()
        if drug:
            gold_ent[("Drug", drug)] += 1
            gold_ent_cls["Drug"] += 1
        for k in ENTITY_KEYS:
            val = m.get(k,'')
            if not val: continue
            for v in (val if isinstance(val, list) else [val]):
                v = v.strip()
                if not v: continue
                lbl = k.upper() if k == "ade" else k.capitalize()
                gold_ent[(lbl, v)] += 1 ; gold_ent_cls[lbl] += 1
                rel_lbl = f"{lbl}-Drug"
                gold_rel[(rel_lbl, v, drug)] += 1
                gold_rel_cls[rel_lbl] += 1

tmp = defaultdict(int)
for (lbl, _, _), cnt in gold_rel.items():
    tmp[(lbl, '', '')] += cnt
gold_rel = Counter(tmp)

# parse all converted .ann files
pred_ent = Counter(); pred_rel = Counter()
for ann in ANN_DIR.glob("*.ann"):
    e_cnt, r_cnt = parse_ann_to_counters(ann)
    pred_ent += e_cnt ; pred_rel += r_cnt

# corpus-level statistics
tot_gold_e = sum(gold_ent.values());   tot_pred_e = sum(pred_ent.values())
tot_gold_r = sum(gold_rel.values());   tot_pred_r = sum(pred_rel.values())

# counters
missing_ent_keys = {}      
add_ent_keys     = {}     
missing_rel_keys = {}    
add_rel_keys     = {}    

# entities
for k, gold_n in gold_ent.items():
    diff = gold_n - pred_ent.get(k, 0)
    if diff > 0:              
        missing_ent_keys[k] = diff

for k, pred_n in pred_ent.items():
    diff = pred_n - gold_ent.get(k, 0)
    if diff > 0:                
        add_ent_keys[k] = diff

# relations
counter = 1
for k, gold_n in gold_rel.items():
    #if counter >1: break
    #counter +=1
    diff = gold_n - pred_rel.get(k, 0)
    if diff > 0:
        missing_rel_keys[k] = diff

for k, pred_n in pred_rel.items():
    diff = pred_n - gold_rel.get(k, 0)
    if diff > 0:
        add_rel_keys[k] = diff

# corpus-level numbers
missing_e = sum(missing_ent_keys.values())
add_e     = sum(add_ent_keys.values())
missing_r = sum(missing_rel_keys.values())
add_r     = sum(add_rel_keys.values())

print("CSV-gold  vs  converted-BRAT (duplicate-aware)")
print(f"Entities gold : {tot_gold_e}")
print(f"Entities ann  : {tot_pred_e}")
print(f"  • missing     : {missing_e}")
print(f"  • additional  : {add_e}\n")
print(f"Relations gold: {tot_gold_r}")
print(f"Relations ann : {tot_pred_r}")
print(f"  • missing     : {missing_r}")
print(f"  • additional  : {add_r}\n")

print("Gold entity totals by class")
for lbl,c in sorted(gold_ent_cls.items()):
    print(f"{lbl:<10}: {c}")

print("Gold relation totals by class")
for lbl,c in sorted(gold_rel_cls.items()):
    print(f"{lbl:<15}: {c}")


CSV-gold  vs  converted-BRAT (duplicate-aware)
Entities gold : 34507
Entities ann  : 31040
  • missing     : 5760
  • additional  : 2293

Relations gold: 23718
Relations ann : 20561
  • missing     : 3157
  • additional  : 0

Gold entity totals by class
ADE       : 778
Dosage    : 2722
Drug      : 10789
Duration  : 436
Form      : 4380
Frequency : 4038
Reason    : 3538
Route     : 3568
Strength  : 4258
Gold relation totals by class
ADE-Drug       : 778
Dosage-Drug    : 2722
Duration-Drug  : 436
Form-Drug      : 4380
Frequency-Drug : 4038
Reason-Drug    : 3538
Route-Drug     : 3568
Strength-Drug  : 4258


In [6]:
pred_rel_cls = Counter()
for (rel_type, _, _), cnt in pred_rel.items():
    pred_rel_cls[rel_type] += cnt

print("\n--- Relation counts: CSV-gold  vs  BRAT ---")
print(f"{'Relation':<18}{'gold':>8}{'ann':>8}{'Δ':>8}{'%Δ':>8}")
print("-"*42)
for rel in sorted(gold_rel_cls):
    gold_n = gold_rel_cls[rel]
    pred_n = pred_rel_cls.get(rel, 0)
    delta  = pred_n - gold_n
    pct    = (delta / gold_n * 100) if gold_n else 0
    print(f"{rel:<18}{gold_n:>8}{pred_n:>8}{delta:>8}{pct:>7.1f}%")



--- Relation counts: CSV-gold  vs  BRAT ---
Relation              gold     ann       Δ      %Δ
------------------------------------------
ADE-Drug               778     486    -292  -37.5%
Dosage-Drug           2722    2495    -227   -8.3%
Duration-Drug          436     316    -120  -27.5%
Form-Drug             4380    4098    -282   -6.4%
Frequency-Drug        4038    3167    -871  -21.6%
Reason-Drug           3538    2514   -1024  -28.9%
Route-Drug            3568    3423    -145   -4.1%
Strength-Drug         4258    4062    -196   -4.6%
